In [ ]:
import cv2
import urllib2
import numpy as np
import sys
import matplotlib.pyplot as plt

from linesearch import *

%matplotlib inline

In [ ]:
host = "192.168.1.85:8080"
hoststr = 'http://' + host + '/?action=stream'
print 'Streaming ' + hoststr
l_img = []

n_row = 6
n_col = 7

# 从摄像头获取 20 张棋盘图，以及一张真实赛道图 （cal0.png）

注意 ret 要为 true

In [ ]:
stream=urllib2.urlopen(hoststr)
bytes=''
while True:
    bytes+=stream.read(1024)
    a = bytes.find('\xff\xd8')
    b = bytes.find('\xff\xd9')
    if a!=-1 and b!=-1:
        break
        
jpg = bytes[a:b+2]
bytes= bytes[b+2:]
img = cv2.imdecode(np.fromstring(jpg, dtype=np.uint8),flags=1)

plt.imshow(img)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = (gray<180).astype(np.uint8)*255
ret, corners = cv2.findChessboardCorners(gray, (n_col,n_row), None)
print(ret)
l_img.append(img)

In [ ]:
plt.imshow(gray, 'gray')

In [ ]:
#cv2.imwrite("./cal0.png", img)

# 校正棋盘图

In [ ]:
objp = np.zeros((n_row*n_col, 3), np.float32)
objp[:,:2] = np.mgrid[0:n_col, 0:n_row].T.reshape(-1,2)
objpoints = []
imgpoints = []
img_size = (480, 640)

for i in range(21):
    img1 = cv2.imread('./calibrateImage/cal%d.png' % (i))
    gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    gray = (gray<180).astype(np.uint8)*255
    ret, corners = cv2.findChessboardCorners(gray, (n_col,n_row), None)
    if ret is True:
        objpoints.append(objp)
        imgpoints.append(corners)
        
ret,mtx,dist,rvecs,tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None, None)

In [ ]:
ret

# 内参

In [ ]:
K = mtx
K

In [ ]:
D = dist
D

# 赛道外参 （cal0.png）

In [ ]:
R,_ = cv2.Rodrigues(rvecs[0])
R

In [ ]:
T = tvecs[0]
T

In [ ]:
x_start = -6
x_end = 14
y_start = -30
y_end = 0
coord_3d = np.float32([
    [x_start, y_start,0],
    [x_start, y_end,  0],
    [x_end,   y_start,0],
    [x_end,   y_end,  0],
]).reshape([-1, 3])


src,jac = cv2.projectPoints(coord_3d, R, T, K, D)
src = src[src[:,0,:].argsort(axis=0)[:,0],0,:]
src = src.reshape([1,4,2])
src.astype(np.int)

# 直线校正

相机参数校正的 src 有问题，用直线校正。

In [ ]:
img = cv2.imread("./cal0.png")
plt.imshow(img)

In [ ]:
img = cv2.imread('cal0.png')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = (gray<180).astype(np.uint8)*255
    
dst = np.array([[
    [10,200],
    [10,0],
    [90,0],
    [90,200]
]]).astype(np.float32)
M    = cv2.getPerspectiveTransform(src, dst)
Minv = cv2.getPerspectiveTransform(dst, src)
img_t = cv2.warpPerspective(gray, M, (100,200), cv2.WARP_INVERSE_MAP)

plt.imshow(img_t, 'gray')

# 真实图片

In [ ]:
src = np.array([[
    [0, 350],
    [240,240],
    [400,240],
    [640,350]
]]).astype(np.float32)

dst = np.array([[
    [10,200],
    [10,0],
    [90,0],
    [90,200]
]]).astype(np.float32)
M    = cv2.getPerspectiveTransform(src, dst)

initParams = [
        np.linspace(0.001, 0.02, 10),
        np.array(list(np.linspace(1.5, 50, 10)) + list(np.linspace(-50, 1.5, 10))),
        np.arange( 40, 50, 2),
        np.arange(-15, 15, 2)
]
refPos = [50, 180] # The position of the top-center of chess-board on the ground in img_t
    
def imageReadFromraspberryPi(hoststr):
    stream=urllib2.urlopen(hoststr)
    bytes=''
    while True:
        bytes+=stream.read(1024)
        a = bytes.find('\xff\xd8')
        b = bytes.find('\xff\xd9')
        if a!=-1 and b!=-1:
            break
        
    jpg = bytes[a:b+2]
    bytes= bytes[b+2:]
    img = cv2.imdecode(np.fromstring(jpg, dtype=np.uint8),flags=1)
    return img


def perspectTransform(img, M_trans):
    # binarize image for white line
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = (gray<180).astype(np.uint8)*255
    
    # perspectTransform binarized image
    img_t = cv2.warpPerspective(gray, M_trans, (100,200), cv2.WARP_INVERSE_MAP)
    img_t[180:,0:10] = 255
    img_t[180:,90:]  = 255
    return img_t


def processImage(hoststr, M_trans, initParams, refPos):
    seedParams = []
    
    while 1:
        img = imageReadFromraspberryPi(hoststr)
        img_t = perspectTransform(img, M_trans)
        paramSearch = None
        if len(seedParams) == 0:
            paramSearch = initParams
        else:
            lla = np.linspace(seedParams[0]-(initParams[0][1]-initParams[0][0]),
                              seedParams[0]+(initParams[0][1]-initParams[0][0]), 3)
            llb = np.linspace(seedParams[1]-(initParams[1][1]-initParams[1][0]),
                              seedParams[1]+(initParams[1][1]-initParams[1][0]), 3)
            llc = np.arange(seedParams[2]-2, seedParams[2]+3, 2)
            lld = np.arange(seedParams[3]-1, seedParams[3]+2)
            paramSearch = [lla, llb, llc, lld]

        coords,params = getBestParams(img_t, paramSearch, refPos)
        score = params[4]
        if score < 50:
            seedParams = []
        else:
            seedParams = params
        
        print(params)

In [ ]:
processImage(hoststr, M, initParams, refPos)

In [ ]:
stream=urllib2.urlopen(hoststr)
bytes=''
while True:
    bytes+=stream.read(1024)
    a = bytes.find('\xff\xd8')
    b = bytes.find('\xff\xd9')
    if a!=-1 and b!=-1:
        break
        
jpg = bytes[a:b+2]
bytes= bytes[b+2:]
img = cv2.imdecode(np.fromstring(jpg, dtype=np.uint8),flags=1)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = (gray<180).astype(np.uint8)*255
dst = np.array([[
    [10,200],
    [10,0],
    [90,0],
    [90,200]
]]).astype(np.float32)
M    = cv2.getPerspectiveTransform(src, dst)
Minv = cv2.getPerspectiveTransform(dst, src)
img_t = cv2.warpPerspective(gray, M, (100,200), cv2.WARP_INVERSE_MAP)
img_t[180:,0:10] = 255.
img_t[180:,90:]  = 255.

plt.imshow(img_t, 'gray')

In [ ]:
initParams = [
        np.linspace(0.001, 0.02, 10),
        np.array(list(np.linspace(1.5, 50, 10)) + list(np.linspace(-50, 1.5, 10))),
        np.arange( 40, 50, 2),
        np.arange(-15, 15, 2)
]
refPos = [50, 180]  
seedParams = []
paramSearch = None

if len(seedParams) == 0:
    paramSearch = initParams
else:
    lla = np.linspace(seedParams[0]-(initParams[0][1]-initParams[0][0]),
                      seedParams[0]+(initParams[0][1]-initParams[0][0]), 3)
    llb = np.linspace(seedParams[1]-(initParams[1][1]-initParams[1][0]),
                      seedParams[1]+(initParams[1][1]-initParams[1][0]), 3)
    llc = np.arange(seedParams[2]-2, seedParams[2]+3, 2)
    lld = np.arange(seedParams[3]-1, seedParams[3]+2)
    paramSearch = [lla, llb, llc, lld]

coords,params = getBestParams(img_t, paramSearch, refPos)


In [ ]:
plt.imshow(img_t, 'gray')


In [ ]:
plt.imshow(img_t, "gray")
for i in range(4):
    plt.plot(coords[i], coords[4])
    
plt.xlim(0, 100)